# Environment Setup

### Install neccessary Library
The libraries include:
- langchain framework'
- GPT4ALL, OpenAI and HuggingFace for various embedding methods and LLMs
- Document loaders
- Dependent libraries

__Note__ : 
- It requires C++ builder for building a dependant library for Chroma. Check out https://github.com/bycloudai/InstallVSBuildToolsWindows for instruction. 
- Python version: 3.12.4
- Pydantic version: 2.7.3. There is issue with pydantic version 1.10.8 

In [ ]:
!pip install --upgrade -r requirements.txt

### Get Environment Parameters
Prepare the list of parameter in .env file for later use. 
Parameters: 
- API keys for LLMs
    - OPENAI_API_KEY 
    - HUGGINGFACEHUB_API_TOKEN 
- Directory / location for documents and vector databases
    - DOC_ARVIX = "./source/from_arvix/"
    - DOC_WIKI = "./source/from_wiki/"
    - VECTORDB_OPENAI_EM = "./vector_db/openai_embedding/"
    - VECTORDB_MINILM_EM = "./vector_db/gpt4all_miniLM/"
    - TS_RAGAS = "./evaluation/testset/by_RAGAS/"
    - TS_PROMPT = "./evaluation/testset/by_direct_prompt/"
    - EVAL_DATASET = "./evaluation/evaluation_data_set/"
    - EVAL_METRIC = "./evaluation/evaluation_metric"


In [29]:
import os
from dotenv import load_dotenv
load_dotenv()

True

# I. Build a simple RAG 

<img src="diagrams/HL architecture.png" alt="HL arc" title= "HL Architecture" />

The system comprises of 5 components: 

- Internal data, documents: The system starts with a collection of internal documents and / or structured databases. Documents can be in text, PDF, photo or video formats. These documents and data are sources for the specified knowledgebase.

- Embedding processor: The documents and database entries are processed to create vector embeddings. Embeddings are numerical representations of the documents in a high-dimensional space that capture their semantic meaning. 

- Vector database: the vectorized chunk of documents and database entries are stored on vector database to be search and retrieved in a later stage. 

- Query processor: The query processor takes the user's query and performs semantic search against the vectorized database. This component ensures that the query is interpreted correctly and retrieves relevant document embeddings from the vectorized DB. It combines the user's original query with the retrieved document embeddings to form a context-rich query. This augmented query provides additional context that can help in generating a more accurate and relevant response.

- LLM: pre-trained large language model where the augmented query is passed to for generating a response based on the query and the relevant documents.

The system involves 2 main pipelines: the embedding pipeline and the retrieval pipeline. Each pipeline has specific stages and processes that contribute to the overall functionality of the system.

In this experiment, we use Langchain as a framework to build a simple RAG as a chain of tasks, which interacts with surrounding services like parsing, embedding, vector database and LLMs 

### Pipeline 1 - Knowledge Embeddings

Pipeline 1: Embedding pipeline is to initiate the vectorized knowledgebase. It can be run whenever the knowledgebase needs to update. 

<img src="diagrams/Pipeline 1 - Knowledge Embedding.png" alt="Pipeline1" title="Pipeline 1 - Embeddings" />

#### Step 1. Loading

In this step, we load data from various sources. Make them ready to ingest.
We will download 5 articles from ARVIX with query "RAG for Large Language Model" and store them locally and ready for next steps of embedding

In [19]:
import arxiv 
client = arxiv.Client()
search = arxiv.Search(
  query = "RAG for Large Language Model",
  max_results = 5,
#  sort_by = arxiv.SortCriterion.SubmittedDate
)

results = client.results(search)
all_results = list(client.results(search)) 

In [20]:
# Print out the articles' titles
for r in all_results:
    print(f"{r.title} {r.entry_id}")

MultiHop-RAG: Benchmarking Retrieval-Augmented Generation for Multi-Hop Queries http://arxiv.org/abs/2401.15391v1
Prompt-RAG: Pioneering Vector Embedding-Free Retrieval-Augmented Generation in Niche Domains, Exemplified by Korean Medicine http://arxiv.org/abs/2401.11246v1
Seven Failure Points When Engineering a Retrieval Augmented Generation System http://arxiv.org/abs/2401.05856v1
The Good and The Bad: Exploring Privacy Issues in Retrieval-Augmented Generation (RAG) http://arxiv.org/abs/2402.16893v1
CLAPNQ: Cohesive Long-form Answers from Passages in Natural Questions for RAG systems http://arxiv.org/abs/2404.02103v1


In [32]:
# Purpose: download articles and save them in pre-defined location for later use
# Prepare: create the environment paramter DOC_ARVIX for the path to save articles. 
# Download and save articles in PDF format to the "RAG_for_LLM" folder under ARVIX_DOC path
DOC_ARVIX = os.getenv("DOC_ARVIX") 
directory_path = os.path.join(DOC_ARVIX,"RAG_for_LLM") 
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
for r in all_results:
    r.download_pdf(dirpath=directory_path)

#### Step 2. Parsing

This step and the previous one are usually processed together. I try to separate them to make attention that these are not always coupled.
We use available library DirectoryLoader and PyMuPDFLoader from Langchain to load and parse all .pdf files in the directory.
We can use corresponding loader for other data types such as excel, presentation, unstructured ... 

Refer to https://python.langchain.com/v0.1/docs/integrations/document_loaders/ for other available loaders. 
We also use the OCR library rapidocr to extract image as text. Certainly, the trade-off is processing time. It took 18 minutes to parse 5 pdf files with OCR compared to 0.1 second without. 

In [51]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyMuPDFLoader
directory_path = os.path.join(DOC_ARVIX,"RAG_for_LLM") 
loader_kwargs = {"extract_images":True} #Use OCR to extract image as text
pdf_loader = DirectoryLoader(
        path=directory_path,
        glob="*.pdf",
        loader_cls=PyMuPDFLoader,
        loader_kwargs=loader_kwargs
    )
pdf_documents = pdf_loader.load()

#### Step 3. Chunking

Divide the data into smaller chunks for better handling, processing, and retrieving.
There is a limitation on number of tokens which the embedding service can process at later stage which requires documents are chunked in smaller size.
There are many of chunking methods from Langchain. In which, Recursive CharacterText and Semantic are most popular. 

Reference: https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/ 

In [54]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
text_chunks = text_splitter.split_documents(pdf_documents)

#### Step 4. Vectorizing

Vectors are semantic representation of texts. 
This is an important step to make documents searchable in the later pipeline. 
Embedding is an essential step in Transformer architecture, underlined to every modern LLMs. Therefore, many LLMs provide their embedding functions as services which are ready to use, e.g. OpenAI embedding API. However, it is important to consider privacy risk when exposing internal data to those services.

IMPORTANT NOTE: 
1. the embedding method to perform similarity search in the retrieval pipeline must be the same to the one used to vectorize documents in this step. 
2. Public embedding method such as OpenAIEmbedding may cost a fraction of money and leak internal data.  

Reference: https://python.langchain.com/v0.1/docs/modules/data_connection/text_embedding/

In [55]:
from langchain_openai.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

#### Step 5. Storing

There are some vector databases of choices: Chroma, FAISS, Pinecone ... 
We will create Chroma vector database with openai embedding method. 

Note: different embedding methods will result different vector dimensions and cannot be stored together. 
The same embedding method to be used in retrieval pipeline

Reference: https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/ 

In [56]:
from langchain.vectorstores import Chroma
persist_directory = os.getenv("VECTORDB_OPENAI_EM")
persist_directory = os.path.join(persist_directory,"RAG_for_LLM")
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

vectordb = Chroma.from_documents(documents=text_chunks,  embedding=embeddings, persist_directory=persist_directory)
vectordb.persist()

c:\Users\derek\OneDrive\1 - Technology\Workspace\rag_win\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


### Pipeline 2 - Retrieving & Generating

Retrieval pipeline is to retrieve relevant chunk of knowledge from pre-prepared vectorized knowledge to enrich the LLM prompt with specified context. This pipeline is run to respond to each user’s query. 

<img src="diagrams/Pipeline 2 - Retrieval.png" alt="Pipeline2" title="Pipeline 2 - Retrieval & Generation" />

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

#### Step 1. Query

In [4]:
user_query = "What is retrieval augmented generation?"
#user_query = "Describe the RAG-Sequence Model?"

#### Step 2. Retrieve

Need to load from store if there is, here is Chroma vectordb we have just persisted. 
Perform a semantic search in the vectorized database to retrieve relevant embedded documents.

NOTE: The embedding method used in this step must be same as which used to vectorize knowledges in the previous pipeline.

There is opportunity to improve efficiency and quality of similarity search, especially when the knowledgebase gets larger and more complicated (type of sources)

In [2]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
db_directory = os.getenv("VECTORDB_OPENAI_EM")
db_directory = os.path.join(db_directory,"RAG_for_LLM")
embeddings = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=db_directory, embedding_function=embeddings)
retriever = vectordb.as_retriever()

In [9]:
retriever.invoke(user_query)

[Document(metadata={'author': '', 'creationDate': "D:20240120233737+09'00'", 'creator': '', 'file_path': 'source\\from_arvix\\RAG_for_LLM\\2401.11246v1.Prompt_RAG__Pioneering_Vector_Embedding_Free_Retrieval_Augmented_Generation_in_Niche_Domains__Exemplified_by_Korean_Medicine.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20240120233737+09'00'", 'page': 1, 'producer': 'Microsoft: Print To PDF', 'source': 'source\\from_arvix\\RAG_for_LLM\\2401.11246v1.Prompt_RAG__Pioneering_Vector_Embedding_Free_Retrieval_Augmented_Generation_in_Niche_Domains__Exemplified_by_Korean_Medicine.pdf', 'subject': '', 'title': 'Microsoft Word - Prompt-GPT_v1', 'total_pages': 26, 'trapped': ''}, page_content='2 \n1. Introduction \nRetrieval-Augmented Generation (RAG) models combine a generative model with an information \nretrieval function, designed to overcome the inherent constraints of generative models.(1) They \nintegrate the robustness of a large language model (LLM) with the relevance and up-t

#### Step 3. Augmented Prompt

There are many ways to write the prompt. It will basically instruct the LLM to generate result based on the {question} and the {context}.

The context is inputted from the retrieved documents from p previous step. 

In [11]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. 
If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [12]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
setup = RunnableParallel(context=retriever, question=RunnablePassthrough())

#### Step 4. Response Generating

We now send the augmented prompt to instruct a LLM generating response to user's query. The response is finally parsed for readable. 
In this experiment, we use OpenAI model GPT3.5-Turbo. 

Note: There are many options for LLMs selection, from public to private, from simple to advance. Privacy, performance and quality should be considered to trade off. 

In [13]:
from langchain_openai.chat_models import ChatOpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

In [14]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [15]:
# Define an chain of tasks
chain = setup | prompt | model | parser

In [16]:
response = chain.invoke(user_query)
response

'Retrieval-augmented generation (RAG) is a technique that combines a generative model with an information retrieval function, integrating external information sources to enhance text generation.'

# II. RAG Evaluation with RAGAS

This framework (RAGAS) is only used for demostration purpose. It is NOT practical when scaling up the test set. Reasons are: 
- Easy to hit run-time errors.
- Exceed TPM limits of the LLMs, esp, OpenAI's ones.
- Quite costly. 
- Not very mature to work with other LLMs than OpenAI's

### Generate synthesis Test Dataset

In [72]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
import tqdm

In [73]:
import os
from dotenv import load_dotenv
load_dotenv()

True

It is important to set the runtime to asynchronous for test set generating. 

In [74]:
import nest_asyncio
nest_asyncio.apply()

Define LLMs to: 
- Generate questions from documents (generator_LLM)
- Generate anwsers (aka ground truth) to questions and documents (critic LLM)

In [75]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0) 
critic_llm = ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

In [76]:

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings,
 #   run_config= RunConfig(max_wait=60)
)

# Change resulting question type distribution
distributions = {
    simple: 0.2,
    multi_context: 0.4,
    reasoning: 0.4
}


Load documents to be used for question generation. This should be the same as documents we used to build vector DB (knowledgebase)

In [77]:
from langchain.document_loaders import ArxivLoader
test_docs = ArxivLoader(query="RAG for Large Language Model",  load_max_docs=5).load()

Below is to generate 5 testset (5 questions, answers / ground truth)

In [79]:

try:
    testset = generator.generate_with_langchain_docs(test_docs, test_size=5, distributions = distributions) 
except Exception as e:
    print (e)

Filename and doc_id are the same for all nodes.                   
Generating: 100%|██████████| 5/5 [05:32<00:00, 66.52s/it] 


Write testset to csv and json for future use

In [87]:
ts = testset.to_pandas()
ts_path = os.getenv("TS_RAGAS")
ts_path = os.path.join(ts_path,"RAG_for_LLM")
if not os.path.exists(ts_path):
    os.makedirs(ts_path)
ts.to_csv(os.path.join(ts_path,"testset_arvix.csv"))
ts.to_json(path_or_buf=os.path.join(ts_path,"testset_arvix.json"),orient='records',lines=True)

### Evaluation with RAGAS

Load testset from csv file.

In [89]:
from datasets import Dataset

ts_path = os.getenv("TS_RAGAS")
ts_path = os.path.join(ts_path,"RAG_for_LLM","testset_arvix.csv")
eval_dataset = Dataset.from_csv(ts_path)

Generating train split: 5 examples [00:00, 425.39 examples/s]


Invoke the RAG chain with questions in testset to get answers. 

In [106]:
import pandas as pd
ans_df = []
for row in eval_dataset:
  question = row["question"]
  answer = chain.invoke(question)
  ans_df.append(
      {"question" : question,
       "answer" : answer,
       "contexts" : [doc.page_content for doc in retriever.get_relevant_documents(question)],
       "ground_truth" : row["ground_truth"]
       }
  )
ans_df = pd.DataFrame(ans_df)
ans_dataset = Dataset.from_pandas(ans_df)

c:\Users\derek\OneDrive\1 - Technology\Workspace\rag_win\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Evaluate the anwsers from RAG chain with 'Faithfulness' and 'answer relevancy' metrics. Here, we are using the critic llm (gpt 4) for evaluation

In [111]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

eval_result = evaluate(
  dataset=ans_dataset,
  metrics=[
      faithfulness,
      answer_relevancy
  ],
  llm=critic_llm,
#    run_config=RunConfig(timeout=300,thread_timeout=300)
)

Evaluating: 100%|██████████| 10/10 [01:06<00:00,  6.67s/it]


In [112]:
import pandas as pd
eval_result_df = eval_result.to_pandas()
pd.set_option("display.max_colwidth", 700)
eval_result_df[["question", "contexts", "answer", "ground_truth","faithfulness","answer_relevancy"]]

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy
0,What are the privacy risks associated with Large Language Models (LLMs) as demonstrated by recent research?,"[large language models: A survey. arXiv preprint\narXiv:2312.10997.\nYangsibo Huang, Samyak Gupta, Zexuan Zhong, Kai\nLi, and Danqi Chen. 2023.\nPrivacy implications\nof retrieval-based language models. arXiv preprint\narXiv:2305.14888.\nDaphne Ippolito, Florian Tramèr, Milad Nasr, Chiyuan\nZhang, Matthew Jagielski, Katherine Lee, Christo-\npher A Choquette-Choo, and Nicholas Carlini. 2022.\nPreventing verbatim memorization in language mod-\nels gives a false sense of privacy. arXiv preprint, without necessitating re-training or fine-tuning of\nthe entire system (Shao et al., 2023; Cheng et al.,\n2023). These unique advantages have positioned\nRAG as a favored approach for a range of pra...",I don't know.,"Recent research has demonstrated that Large Language Models (LLMs) are prone to memorizing and inadvertently revealing information from their pre-training corpora, which poses privacy risks. Notably, studies have shown that LLMs can recall and reproduce segments of their training data, and various factors such as model size, data duplication, and prompt length can increase the risk of such memorization.",0.0,0.000000
1,"Given the RAG system's flaws like content gaps, ranking, context, extraction mistakes, format issues, and specificity, plus research areas like chunking, embeddings, and fine-tuning, what strategies could improve query precision and relevance?","[such as tables, figures, formulas, etc. Chunk embeddings are typ-\nically created once during system development or when a new\ndocument is indexed. Query preprocessing significantly impacts\na RAG system’s performance, particularly handling negative or\nambiguous queries. Further research is needed on architectural pat-\nterns and approaches [5] to address the inherent limitations with\nembeddings (quality of a match is domain specific).\n6.2\nRAG vs Finetuning, case studies including an empirical investigation involving 15,000\ndocuments and 1000 questions. Our findings provide a guide to\npractitioners by presenting the challenges faced when implement-\ning RAG systems. We also inclu...","Semantic search technologies can improve query precision and relevance by scanning large databases of information and retrieving data more accurately. These technologies can map questions to relevant documents and return specific text instead of search results, providing more context to the Language Model (LLM). Additionally, utilizing techniques such as document chunking, word embeddings, and knowledge base preparation can enhance the quality of the RAG payload by generating semantically relevant passages and token words ordered by relevance. This approach reduces the need for manual data preparation and addresses issues such as content gaps, ranking, context, extraction mistakes, forma...",The answer to given question is not present in context,0.0,0.882363
2,How does MultiHop-RAG improve LLMs' multi-doc reasoning over current RAG systems?,"[concern that LLM responses might rely on training\nknowledge rather than reasoning from the retrieved\nknowledge base.\n6\nConclusion\nIn this work, we introduce MultiHop-RAG, a novel\nand unique dataset designed for queries that re-\nquire retrieval and reasoning from multiple pieces\nof supporting evidence. These types of multi-hop\nqueries represent user queries commonly encoun-\ntered in real-world scenarios. MultiHop-RAG con-\nsists of a knowledge base, a large collection of, MultiHop-RAG: Benchmarking Retrieval-Augmented Generation for\nMulti-Hop Queries\nYixuan Tang and Yi Yang\nHong Kong University of Science and Technology\n{yixuantang,imyiyang}@ust.hk\nAbstract\nRetrieval-augm...",I don't know.,"MultiHop-RAG improves LLMs' multi-doc reasoning by providing a dataset that consists of a knowledge base, a large collection of multi-hop queries, their ground-tr

The evaluation result of faithfulness is 0 for all questions, even with "I don't know" answers. It seems the RAGAS evaluation is not accurate in this case. 

Write the evaluation result in CSV & Json for future analysis

In [114]:
eval_dataset_path = os.getenv("EVAL_DATASET")
eval_result_path = os.getenv("EVAL_METRIC")

eval_dataset_path = os.path.join(eval_dataset_path,"RAG_for_LLM_Simple_RAG")
eval_result_path = os.path.join(eval_result_path,"RAG_for_LLM_Simple_RAG")

if not os.path.exists(eval_dataset_path):
    os.makedirs(eval_dataset_path)
if not os.path.exists(eval_result_path):
    os.makedirs(eval_result_path)

ans_df.to_csv(os.path.join(eval_dataset_path,"eval_dataset_arvix.csv"))
ans_df.to_json(path_or_buf=os.path.join(eval_dataset_path,"eval_dataset_arvix.json"),orient='records',lines=True)

eval_result_df.to_csv(os.path.join(eval_result_path,"eval_result_arvix.csv"))
eval_result_df.to_json(path_or_buf=os.path.join(eval_result_path,"eval_result_arvix.json"),orient='records',lines=True)

# III. Improved RAG applications and Evaluation

In this section, we are going to apply various methods to improve quality and mitigate failure points of RAG application then evaluate them. 

There is an issue with Chroma that a connection need to be initiated from Notebook. 

In [1]:
# Just to ensure we load environment parameters for each section so that it can run independently
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
tempVDB = Chroma(persist_directory=os.path.join(os.getenv("VECTORDB_OPENAI_EM"),"RAG_for_LLM"), embedding_function=OpenAIEmbeddings())

In [8]:
import Agent
import prompt_collection as p

rag1 = Agent.RAGAgent(
    name = "RAG 1 - Simple RAG",
    model = Agent.GPT_3_5_TURBO,
    vectordb_name="CHROMA_OPENAI_RAG_FOR_LLM",
    rag_type= "SIMPLE_QUESTION_ANSWER_RAG"
)

### Create Testset

In [1]:
import evaluator as eval

testset = eval.generate_testset(eval.ARVIX_RAG_FOR_LLM)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
c:\Users\derek\OneDrive\1 - Technology\Workspace\rag_win\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\derek\.cache\huggingface\token
Login successful


  1%|          | 1/89 [00:01<01:48,  1.24s/it]

Question 1 : Question: What is the name of the conference where the paper was presented?
Context 1 : Seven Failure Points When Engineering a Retrieval Augmented
Generation System
Scott Barnett, Stefanus Kurniawan, Srikanth Thudumu, Zach Brannelly, Mohamed Abdelrazek
{scott.barnett,stefanus.kurniawan,srikanth.thudumu,zach.brannelly,mohamed.abdelrazek}@deakin.edu.au
Applied Artificial Intelligence Institute
Geelong, Australia
ABSTRACT
Software engineers are increasingly adding semantic search capabil-
ities to applications using a strategy known as Retrieval Augmented
Generation (RAG). A RAG system involves finding documents that
semantically match a query and then passing the documents to a
large language model (LLM) such as ChatGPT to extract the right
answer using an LLM. RAG systems aim to: a) reduce the problem
of hallucinated responses from LLMs, b) link sources/references
to generated responses, and c) remove the need for annotating
documents with meta-data. However, RAG systems s

  2%|▏         | 2/89 [00:03<02:23,  1.65s/it]

...

 99%|█████████▉| 88/89 [02:22<00:00,  1.11it/s]

Question 88 : Question: What was Glumdalclitch's age when she took care of Gulliver?
Context 88 : Question: who was glumdalclitch how did she help gulliver
Passage 1: When the Queen of Brobdingnag takes Gulliver into her court , he has Glumdalclitch brought to
court with him . The prideful Gulliver thinks of himself as being greatly honored and promoted by moving
to court , but never ceases to love and seek the approval of the little girl who first helped him . Indeed , he
remembers her fondly even after returning to England .
Passage 2: Glumdalclitch is the name Gulliver gives his “ nurse ” in Book II of Jonathan Swift ’s Gulliver ’s
Travels . In Book I , Gulliver travels to the land of Lilliput . Leaving there , he travels to the land of Brobdingnag
. In Lilliput , Gulliver was a giant , and in Brobdingnag , he is a dwarf , with the proportions reversed .
Passage 3: This article is written like a personal reflection or opinion essay that states a Wikipedia editor ’s
personal feelings

100%|██████████| 89/89 [02:22<00:00,  1.60s/it]
WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


Question 89 : Question: What percentage of its oil did Japan depend on the United States for?
Context 89 : Conversation
User: why did the us demand trade with japan
Passages
Passage 1
The United States reacted by seeking to bring the Japanese war effort to a complete halt by imposing a full
embargo on all trade between the United States to Japan on 1 August 1941 , demanding that Japan withdraw
all troops from both China and Indochina . Japan was dependent on the United States for 80 percent of its oil ,
resulting in an economic and military crisis for Japan that could not continue its war effort with China without
access to petroleum and oil products . Attack
Passage 2
The U.S. embargoes gave Japan a sense of urgency . It would either have to agree to Washington ’s demands or
use force to gain access to the resources it needed .
Passage 3
Japan ’s goal after 1931 was economic dominance of most of East Asia , often expressed in Pan-Asian terms
of “ Asia for the Asians . ” . Japan was de

100%|██████████| 89/89 [01:19<00:00,  1.13it/s]


In [2]:
testset

,context,question,answer
0,Seven Failure Points When Engineering a Retrie...,Question: What is the name of the conference w...,Answer: 3rd International Conference on AI Eng...
1,"CAIN 2024, April 2024, Lisbon, Portugal\nScott...",Question: What is the name of the research dir...,Answer: A research direction for RAG systems b...
2,Seven Failure Points When Engineering a Retrie...,Question: What is the total number of question...,Answer: 1000
3,"CAIN 2024, April 2024, Lisbon, Portugal\nScott...",Question: What are the key considerations when...,Answer: The key considerations when engineerin...
4,Seven Failure Points When Engineering a Retrie...,Question: What is the number of documents invo...,"Answer: 15,000"
...,...,...,...
84,Question: where would a commercial quantity of...,Question: Where would a commercial quantity of...,Answer: Nuclear reactors.
85,Question: where are nimbus clouds found in the...,Question: At what altitude are nimbostratus cl...,Answer: from near surface in the low levels to...
86,Question: who was glumdalclitch how did she he...,Question: What was Glumdalclitch's occupation ...,Answer: Glumdalclitch's occupation or skill wa...
87,Question: who was glumdalclitch how did she he...,Question: What was Glumdalclitch's age when sh...,Answer: 9 years old.


### Evaluate RAG1

In [1]:
# Just to ensure we load environment parameters for each section so that it can run independently
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
tempVDB = Chroma(persist_directory=os.path.join(os.getenv("VECTORDB_OPENAI_EM"),"RAG_for_LLM"), embedding_function=OpenAIEmbeddings())

import Agent
import prompt_collection as p

rag1 = Agent.RAGAgent(
    name = "RAG 1 - Simple RAG",
    model = Agent.GPT_3_5_TURBO,
    vectordb_name="CHROMA_OPENAI_RAG_FOR_LLM",
    rag_type= "SIMPLE_QUESTION_ANSWER_RAG"
)

In [2]:
import evaluator as eval

result = eval.rag_evaluate(rag1)

c:\Users\derek\OneDrive\1 - Technology\Workspace\rag_win\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/89 [00:00<?, ?it/s]

Question 1 : Question: What is the name of the conference where the paper was presented? 
answer 1 : The name of the conference where the paper was presented is "Joint Meeting on European Software Engineering Conference and Symposium on the Foundations of Software Engineering". 


  2%|▏         | 2/89 [00:03<02:09,  1.49s/it]

Question 2 : Question: What is the name of the research direction that the authors propose for RAG systems based on the lessons learned from the three case studies? 
answer 2 : The research direction proposed for RAG systems based on the lessons learned from the three case studies is not explicitly mentioned in the provided context. 


  3%|▎         | 3/89 [00:04<01:51,  1.29s/it]

...

answer_relevancy 87 : Let's evaluate this answer. 

Total rating: 8.5
Question 88 : Question: What was Glumdalclitch's age when she took care of Gulliver?
answer 88 : Glumdalclitch was nine years old when she took care of Gulliver.
ground_truth 88 : Answer: 9 years old.


 99%|█████████▉| 88/89 [00:37<00:00,  2.59it/s]

answer_relevancy 88 : Now, please provide your rating.
Question 89 : Question: What percentage of its oil did Japan depend on the United States for?
answer 89 : Japan depended on the United States for 80 percent of its oil.
ground_truth 89 : Answer: 80 percent


100%|██████████| 89/89 [00:37<00:00,  2.36it/s]

answer_relevancy 89 : Please provide your rating.


In [3]:
result.to_pandas()

,question,answer,ground_truth,contexts,answer_relevancy
0,Question: What is the name of the conference w...,The name of the conference where the paper was...,Answer: 3rd International Conference on AI Eng...,[Joint Meeting on European Software Engineerin...,Please provide your rating.
1,Question: What is the name of the research dir...,The research direction proposed for RAG system...,Answer: A research direction for RAG systems b...,[• A research direction for RAG systems based ...,Please provide your rating.
2,Question: What is the total number of question...,The total number of questions in the BioASQ da...,Answer: 1000,[The previous case studies focused on document...,Please provide your rating.
3,Question: What are the key considerations when...,The key considerations when engineering a RAG ...,Answer: The key considerations when engineerin...,[identify the patterns.\n• What are the key co...,Total rating: 2.0
4,Question: What is the number of documents invo...,I don't know.,"Answer: 15,000",[Num. of Evidence Needed\nCount\nPercentage\n0...,Please provide your rating.
...,...,...,...,...,...
84,Question: Where would a commercial quantity of...,A commercial quantity of cobalt-60 would typic...,Answer: Nuclear reactors.,[Question: where would a commercial quantity o...,Please provide your rating.
85,Question: At what altitude are nimbostratus cl...,Nimbostratus clouds are typically found in the...,Answer: from near surface in the low levels to...,[Question: where are nimbus clouds found in th...,Please provide your rating.
86,Question: What was Glumdalclitch's occupation ...,Glumdalclitch's occupation or skill was being ...,Answer: Glumdalclitch's occupation or skill wa...,[Question: who was glumdalclitch how did she h...,Let's evaluate this answer. \n\nTotal rating: 8.5
87,Question: What was Glumdalclitch's age when sh...,Glumdalclitch was nine years old when she took...,Answer: 9 years old.,[Question: who was glumdalclitch how did she h...,"Now, please provide your rating."
